In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Run this code for different raw data files
data=pd.read_csv("../Data/Raw data/July18_myspeed.csv") #To read csv file
df=pd.DataFrame(data)

df1=pd.DataFrame()
df['signal_strength']=pd.to_numeric(df['signal_strength'],errors='coerce') #This part is for March18_myspeed.csv file only to convert string type data(signal strength) to float64 type
df['speed']=df['speed']*1000 #This part is for March18_myspeed.csv file only to convert Mbps to Kbps
df['download/upload']=df['download/upload'].replace('Upload','upload') #This part is for March18_myspeed.csv file only to replace Upload to upload
df['download/upload']=df['download/upload'].replace('Download','download') #This part is for March18_myspeed.csv file only to replace Download to download

FileNotFoundError: [Errno 2] File ../Data/Raw data/July18_myspeed.csv does not exist: '../Data/Raw data/July18_myspeed.csv'

In [14]:
df.dropna(axis=0,subset=['state'],inplace=True,how='any') #To drop rows whose state(area) is unkown
df=df[df['speed']>0] #To remove rows with data speeds are negative

In [15]:
df['signal_strength'] = df['signal_strength'].replace(np.nan,0) 

In [16]:
#Random sampling of data to 5% of particular type
def sample(df,df1):
    state = df['state'].unique()
    serviceprov = df['service_provider'].unique()
    tech = df['technology'].unique()
    d_u = df['download/upload'].unique()
    df1=pd.DataFrame()
    for k1 in state:
        for k2 in serviceprov:
            for k3 in tech:
                for k4 in d_u:
                    a=((df['state']==k1) & (df['service_provider']==k2) & (df['technology']==k3) & (df['download/upload']==k4))
                    df1=df1.append(df[a].sample(frac =0.05))
    return df1
df2=sample(df,df1)

In [19]:
def replacena(df2):
    state = df['state']
    serviceprov = df['service_provider']
    tech = df['technology']
    d_u = df['download/upload']
    speed = df['speed']
    strength = df['signal_strength']
    for key,value in strength.iteritems():
        if(value == 0):
            tstate = df.loc[key]['state']
            tservice = df.loc[key]['service_provider']
            ttech = df.loc[key]['technology']
            tdu = df.loc[key]['download/upload']
            a=((df2['state']==tstate) & (df2['service_provider']==tservice) & (df2['technology']==ttech) & (df2['download/upload']==tdu))
            df2[a]=df2[a].replace(0,np.nan)
            df2[a]=df2[a].fillna(df2[a]['signal_strength'].mean())

In [20]:
replacena(df2) #Replacing NA value with average value of that particular state(area),technology,service provider,download or upload type

In [21]:
df2['Month'] = 'July' #add respective month which we are working on
df2['Year'] = 2018 #add respective year which we are working on
df2.to_csv("Sampled_Cleaned_July181.csv", index=False) # saving that data to a csv file